<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/feature%2Fadd-multi-task-learning/multi_task_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>